In [1]:
# Instalar as bibliotecas
!pip install -q earthengine-api geemap==0.16.0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
# Importar as bibliotecas
import geemap
import numpy as np
import geopandas as gpd
import ee
import json

In [3]:
# Autenticação
ee.Authenticate()
ee.Initialize(project='ee-gustavoohs')

In [4]:
# Lendo AOI
gdf = gpd.read_file('/content/drive/MyDrive/Pós AmbGEO/AOI_DF.shp')
gdf.head()

,MINX,MINY,MAXX,MAXY,CNTX,CNTY,AREA,PERIM,HEIGHT,WIDTH,geometry
0,160665.0,8242285.0,206265.0,8279275.0,183465.0,8260780.0,1.686744e+09,165180.0,36990.0,45600.0,"POLYGON ((-48.16826 -15.87534, -48.1631 -15.54..."


In [5]:
# Converter para json
df_json = json.loads(gdf.to_json())
print(df_json)

{'type': 'FeatureCollection', 'features': [{'id': '0', 'type': 'Feature', 'properties': {'MINX': 160664.99999999997, 'MINY': 8242285.0, 'MAXX': 206264.99999999997, 'MAXY': 8279275.0, 'CNTX': 183464.99999999997, 'CNTY': 8260780.0, 'AREA': 1686744000.0, 'PERIM': 165180.0, 'HEIGHT': 36990.0, 'WIDTH': 45600.0}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-48.168264200104346, -15.875343907527848], [-48.163102477110684, -15.541443341722573], [-47.73841908677864, -15.547130595540853], [-47.74289007040111, -15.881159531869113], [-48.168264200104346, -15.875343907527848]]]}}]}


In [6]:
# Selecionando coordenadas
df_coord = df_json['features']

In [7]:
# Convertendo para FeatureCollection
aoi = ee.FeatureCollection(df_coord)

In [16]:
# Chamando a coleção de imagens

collection = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL') \
    .filter(ee.Filter.date('2024-01-01', '2024-12-31')) \
    .filterBounds(aoi)\
    .first()

In [17]:
# Kmeans
training = collection.sample(
    region=aoi,
    scale=10,
    numPixels=5000,
    seed=0,  # Semente para reprodutibilidade
    geometries=True
)

n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)
classified_image = collection.cluster(clusterer)

In [19]:
# visualização
vis_params = {
    'min': 0,
    'max': n_clusters - 1,
    'palette': ['red', 'green', 'blue', 'yellow', 'purple']
}

mapa = geemap.Map()
mapa.add_basemap('SATELLITE')
mapa.centerObject(aoi, 10)
mapa.addLayer(classified_image, vis_params, 'Clustered Image')
mapa

Map(center=[-15.711371982219736, -47.953157347938266], controls=(WidgetControl(options=['position', 'transpare…

In [21]:
mapa = geemap.Map()
mapa.add_basemap('SATELLITE')
mapa.centerObject(aoi, 10)
mapa.addLayer(collection)
mapa

Map(center=[-15.711371982219736, -47.953157347938266], controls=(WidgetControl(options=['position', 'transpare…